# Time Series Demo: DCorr-X and MGC-X
In this notebook, we demonstrate the cross-distance correlation (`DCorrX`) test and the multiscale graph correlation for time series (`MGCX`) test for independence of time series.

In [6]:
%matplotlib inline
import numpy as np
from scipy.stats import chi2
import matplotlib.pyplot as plt
import random

from mgcpy.independence_tests.dcorrx import DCorrX
from mgcpy.independence_tests.mgcx import MGCX
from mgcpy.benchmarks.ts_benchmarks import IndependentAR1, CorrelatedAR1, NonlinearLag1

**Function to compute and print DCorrX nd MGCX output values, given X and Y:**

In [7]:
def compute_dcorrx(X, Y, max_lag):
    dcorrx = DCorrX(max_lag = max_lag, which_test = 'unbiased')
    dcorrx_statistic, metadata = dcorrx.test_statistic(X, Y)
    p_value, _ = dcorrx.p_value(X, Y)
    optimal_lag = metadata['optimal_lag']

    print("DCorrX test statistic:", dcorrx_statistic)
    print("P Value:", p_value)
    print("Optimal Lag:", optimal_lag)

def compute_mgcx(X, Y, max_lag, is_fast = False):
    mgcx = MGCX(max_lag = max_lag)
    mgcx_statistic, metadata = mgcx.test_statistic(X, Y)
    p_value, _ = mgcx.p_value(X, Y, is_fast = is_fast)
    optimal_lag = metadata['optimal_lag']
    optimal_scale = metadata['optimal_scale']
    
    print("MGCX test statistic:", mgcx_statistic)
    print("P Value:", p_value)
    print("Optimal Lag:", optimal_lag)
    print("Optimal Scale:", optimal_scale)

In [10]:
# Simulate data. X and Y are n-by-1 matrices. Should generally not reject, as X and Y are independent.
n = 40
process = IndependentAR1()

X, Y = process.simulate(n)

max_lag = 0
compute_dcorrx(X, Y, max_lag)
compute_mgcx(X, Y, max_lag, is_fast = True)

DCorrX test statistic: 0.008673431711540155
P Value: 0.421
Optimal Lag: 0
MGCX test statistic: 0.009648195743383858
P Value: 1.0
Optimal Lag: 0
Optimal Scale: [40, 40]


**Example: simulate data `X` and `Y` in the form of `n`-by-`p` and `n`-by-`q` matrices respectively, where `n` is the sample size.** In the following cells, we simulate different time series processes and estimate the power of the test at varying choices of sample size. Additionally, we compare against the Ljung-Box test of correlation.

**Functions for plotting univarite time series and power curves for each of the processes.**

The following are helper functions.

In [16]:
def _compute_power(test, X_full, Y_full, num_sims, alpha, n):
    """
    Helper method estimate power of a test on a given simulation.

    :param test: Test to profile, either DCorrX or MGCX.
    :type test: TimeSeriesIndependenceTest

    :param X_full: An ``[n*num_sims]`` data matrix where ``n`` is the highest sample size.
    :type X_full: 2D ``numpy.array``

    :param Y_full: An ``[n*num_sims]`` data matrix where ``n`` is the highest sample size.
    :type Y_full: 2D ``numpy.array``

    :param num_sims: number of simulation at each sample size.
    :type num_sims: integer

    :param alpha: significance level.
    :type alpha: float

    :param n: sample size.
    :type n: integer

    :return: returns the estimated power.
    :rtype: float
    """
    num_rejects = 0.0

    def worker(s):
        X = X_full[range(n), s]
        Y = Y_full[range(n), s]

        p_value, _ = test['object'].p_value(X, Y, is_fast = test['is_fast'], subsample_size = test['subsample_size'])
        if p_value <= alpha:
            return 1

        return 0

    num_rejects = np.sum(Parallel(n_jobs=-2, verbose=10)(delayed(worker)(s) for s in range(num_sims)))

    return num_rejects / num_sims

def _plot_power(tests, sample_sizes, alpha, process):
    """
    Helper method to generate power curves for time series.

    :param tests: An array-like object containing TimeSeriesIndependenceTest objects.
    :type tests: 1-D array-like

    :param sample_sizes: range of sample sizes for which to estimate power.
    :type sample_sizes: 1-D array-like

    :param alpha: significance level.
    :type alpha: float

    :param process: A TimeSeriesProcess object for which to profile the test.
    :type process: TimeSeriesProcess
    """
    plt.rcParams.update({'font.size': 15})
    fig, ax = plt.subplots()
    plt.title(process.name)
    plt.xlabel("n")
    plt.ylabel("Rejection Probability")
    plt.ylim((-0.05, 1.05))

    for test in tests:
        plt.plot(sample_sizes, test['powers'], linestyle = '-', color = test['color'])
    ax.legend([test['name'] for test in tests], loc = 'upper left', prop={'size': 12})

    ax.axhline(y = alpha, color = 'black', linestyle = '--')
    plt.show()

Similarly, the following are used in the rest of the demo to visualize the simulations.

In [17]:
def plot_1d_ts(X, Y, title, xlab = "X_t", ylab = "Y_t"):
    """
    Method to plot univariate time series on the same figure.

    :param X: An ``[n*1]`` data matrix where ``n`` is the sample size.
    :type X: 2D ``numpy.array``

    :param Y: An ``[n*1]`` data matrix where ``n`` is the sample size.
    :type Y: 2D ``numpy.array``

    :param title: Plot title.
    :type title: string

    :param xlab: x-axis label.
    :type xlab: string

    :param ylab: y-axis label.
    :type ylab: string
    """
    n = X.shape[0]
    t = range(1, n + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7.5))
    fig.suptitle(title)
    plt.rcParams.update({'font.size': 15})

    ax1.plot(t, X)
    ax1.plot(t, Y)
    ax1.legend(['X_t', 'Y_t'], loc = 'upper left', prop={'size': 12})
    ax1.set_xlabel("t")

    ax2.scatter(X,Y, color="black")
    ax2.set_ylabel(ylab)
    ax2.set_xlabel(xlab)

# Power computation functions.

def power_curve(tests, process, num_sims, alpha, sample_size, verbose = False):
    """
    Method to generate power curves for time series.

    :param tests: An array-like object containing TimeSeriesIndependenceTest objects.
    :type tests: 1-D array-like

    :param process: A TimeSeriesProcess object for which to profile the test.
    :type process: TimeSeriesProcess

    :param num_sims: number of simulation at each sample size.
    :type num_sims: integer

    :param alpha: significance level.
    :type alpha: float

    :param verbose: whether to display output.
    :type verbose: boolean

    :param sample_sizes: range of sample sizes for which to estimate power.
    :type sample_sizes: 1-D array-like
    """
    # Store simulate processes.
    n_full = sample_sizes[len(sample_sizes) - 1]
    X_full = np.zeros((n_full, num_sims))
    Y_full = np.zeros((n_full, num_sims))
    for s in range(num_sims):
        X_full[:, s], Y_full[:, s] = process.simulate(n_full)

    for test in tests:
        powers = np.zeros(len(sample_sizes))
        for i in range(len(sample_sizes)):
            n = sample_sizes[i]
            if verbose: print("Estimating power at sample size: %d" % n)
            powers[i] = _compute_power(test, X_full, Y_full, num_sims, alpha, n)
        test['powers'] = powers

    # Display.
    _plot_power(tests, sample_sizes, alpha, process)

def plot_optimal_lags(optimal_lags, process, test_name, color, true_correlations = None, savefig = True):
    """
    Visualize distribution of optimal lag estimates. If the time series process is linear, then plot true cross correlations.

    :param optimal_lags: An array-like object containing optimal lag estimates from each simulation.
    :type optimal_lags: 1-D array-like

    :param process: A TimeSeriesProcess object for which to profile the test.
    :type process: TimeSeriesProcess
    
    :param test_name: String for the filename, either 'mgcx' or 'dcorrx'.
    :type test_name: string

    :param color: Color of histogram bars. Typically red for 'mgcx' and blue for 'dcorrx'.
    :type color: string

    :param true_correlations: Array of size ``max_lag`` + 1, containing the cross correlation of ``X_{t}`` and ``Y_{t-j}`` for each ``j``.
    :type true_correlations: 1-D array-like

    :param savefig: Whether to save the figure. Defaults to True.
    :type savefig: boolean
    """
    plt.rcParams.update({'font.size': 14})
    
    plt.xlabel('Lag j')
    if true_correlations is not None:
        # True correlations of X_{t} and Y_{t-j}at various lags.
        plt.ylabel("Corr(X(t), Y(t-j)) / Freq. of Optimal Lag Estimates")

        j = range(true_correlations.shape[0])
        markerline, stemlines, baseline = plt.stem(j, true_correlations, '-k')
        plt.setp(baseline, 'color', 'k', 'linewidth', 1)
        plt.setp(markerline, 'markerfacecolor', 'k')
    else:
        plt.ylabel("Freq. of Optimal Lag Estimates")
    
    # Optimal lab predictions.
    weights = np.ones_like(optimal_lags)/float(len(optimal_lags))
    plt.hist(optimal_lags, 
             bins = np.arange(len(true_correlations))-0.5, 
             weights = weights, 
             align = 'mid',
             edgecolor ='black',
             color = color)
    
    plt.title(process.title)
    if savefig:
        filename = "optimal_lags_%s_%s.pdf" % (process.filename, test_name)
        plt.savefig(filename)
    plt.show()

**Power as a function of `n` of the tests on each process.** Probability of correctly rejecting the null hypothesis that the time series are independent.

**Full experiment, simulating the above time series, and counting the rejections and optimal lag detections by each test (as a function of `n`).**

In [ ]:
max_lag = 1
num_sims = 35
alpha = 0.05
num_bootstrap = 100 # TO DO: Incorporate this parameter

dcorrx = DCorrX(max_lag = max_lag)
mgcx = MGCX(max_lag = max_lag)

tests = [
    {
        'name' : 'DCorr-X',
        'filename' : 'dcorrx',
        'is_fast' : False,
        'subsample_size' : -1,
        'object' : dcorrx,
        'color' : 'red',
    },
    {
        'name' : 'MGC-X',
        'filename' : 'mgcx',
        'is_fast' : False,
        'object' : mgcx,
        'color' : 'orange',
    },
]

In [ ]:
# Combining all time series simulations.
sample_sizes = range(100, 300, 30)
process = IndependentAR1()
power_curve(tests, process, num_sims, alpha, sample_sizes)

In [ ]:
for p in ['nonlin_dep_lag_m']:
    mgcx_powers = np.genfromtxt('mgcx_' + p + '.csv', delimiter=',')
  
    process = processes[p]
    sample_sizes = mgcx_powers[:, 0]
    powers = {'dcorrx': dcorrx_powers[:, 1], 
              'mgcx': mgcx_powers[:, 1]}
    power_curve(sample_sizes, powers, alpha, process['name'])

**Analyzing the distribution of optimal lags returned by the tests.** We see how that the optimal lags returned by the test corresponds with the true dependence structure of the simulated time series.

In [ ]:
def lin_dep_ar3(n, phi_1 = 0.5, phi_3 = 0.5, sigma2 = 1.0):
    # X_t and Y_t are together a bivarite AR(3).
    # Innovations follow N(0, sigma2).
    
    # Innovations.
    epsilons = np.random.normal(0.0, sigma2, n)
    etas = np.random.normal(0.0, sigma2, n)
    
    X = np.zeros(n)
    Y = np.zeros(n)
    for s in range(3):
        X[s] = epsilons[s]
        Y[s] = etas[s]
    
    # AR(1) process.
    for t in range(3,n):
        X[t] = phi_1*Y[t-1] + phi_3*Y[t-3] + epsilons[t]
        Y[t] = phi_1*X[t-1] + phi_3*X[t-3] + etas[t]
        
    return X, Y

In [ ]:
# Correlated AR(3)
phi_1 = 0.1
phi_3 = 0.8
sigma2 = 1.0
M = 10
num_sims = 100

# Determine true correlations.
rho_XY = np.zeros(M+1)
rho_X = np.zeros(M+1)
rho_X[0] = 1
rho_XY[1] = phi_1 / (1 - phi_3*(phi_1 + phi_3))
rho_X[2] = (phi_1 + phi_3)*rho_XY[1]
for j in range(3, M+1):
    if j%2:
        rho_XY[j] = phi_1*rho_X[j-1] + phi_3*rho_X[j-3]
    else:
        rho_X[j] = phi_1*rho_XY[j-1] + phi_3*rho_XY[j-3]

dcorrx = DCorrX(max_lag = M)
mgcx = MGCX(max_lag = M)
true_correlations = rho_XY
optimal_lags_dcorrx = np.zeros(num_sims)
optimal_lags_mgcx = np.zeros(num_sims)

# Run experiments.
for n in [15, 30, 60]:
    for t in range(num_sims):
        X, Y = lin_dep_ar3(n, phi_1 = phi_1, phi_3 = phi_3, sigma2 = sigma2)
        test_statistic, metadata = dcorrx.test_statistic(X, Y)
        optimal_lags_dcorrx[t] = metadata['optimal_lag']
        test_statistic, metadata = mgcx.test_statistic(X, Y)
        optimal_lags_mgcx[t] = metadata['optimal_lag']
    title_dcorrx = "Dcorr-X, Crosscorrelated AR(3), n = %d" % n
    title_mgcx = "MGC-X, Crosscorrelated AR(3), n = %d" % n
    opt_lag_dist_stems(optimal_lags_dcorrx, true_correlations, title = title_dcorrx, color = 'blue')
    opt_lag_dist_stems(optimal_lags_mgcx, true_correlations, title = title_mgcx, color = 'red')

**Simulate a nonlinear process, estimate the Cross Distance Correlation Function, and show that the optimal lag is correct for large n.**

In [ ]:
sigma2 = 1.0
M = 10
m = 3
num_sims = 100

dcorrx = DCorrX(max_lag = M)
mgcx = MGCX(max_lag = M)
optimal_lags_dcorrx = np.zeros(num_sims)
optimal_lags_mgcx = np.zeros(num_sims)

# Run experiments.
for n in [15, 30, 60]:
    for t in range(num_sims):
        X, Y = nonlin_dep_lag_m(n, m, phi_m = 0.8, sigma2 = 1.0)
        test_statistic, metadata = dcorrx.test_statistic(X, Y)
        optimal_lags_dcorrx[t] = metadata['optimal_lag']
        test_statistic, metadata = mgcx.test_statistic(X, Y)
        optimal_lags_mgcx[t] = metadata['optimal_lag']
    title_dcorrx = "Dcorr-X, Nonlinearly Dependent at Lag 3, n = %d" % n
    title_mgcx = "MGC-X, Nonlinearly Dependent at Lag 3, n = %d" % n
    opt_lag_dist(optimal_lags_dcorrx, title = title_dcorrx, color = 'blue')
    opt_lag_dist(optimal_lags_mgcx, title = title_mgcx, color = 'red')